## Data split

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
from scripts.baseline_model import MoleculeModel
from collections import defaultdict
from tqdm.notebook import tqdm

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('../data/lexa_preprocessed.csv')

In [5]:
data

,smiles,active
0,CC(=O)C1=CC2=CC=CC=C2OC1=O,1
1,C1COC2=CC=CC=C2C1O,1
2,C1=CC=C2C(=C1)C(=CC(=O)O2)O,1
3,C1=CC(=CC=C1C2=COC3=C(C2=O)C=CC(=C3O)O)O,1
4,C1CC1NC2=C3C(=NC(=N2)N)N(C=N3)[C@@H]4C[C@@H](C...,1
...,...,...
299,C1CC(CCC1CN)C(=O)O,0
300,CN(C)CC(C1=CC=C(C=C1)OC)C2(CCCCC2)O,0
301,C1C[C@@H](O[C@@H]1CO)N2C=CC(=NC2=O)N,0
302,CC(=O)N[C@@H]1[C@H](C=C(O[C@H]1[C@@H]([C@@H](C...,0


In [6]:
TARGET_COLUMN = 'active'

In [8]:
features = defaultdict(list)
for smi in tqdm(data.smiles):
    mol = MoleculeModel(smi, count=False)
    representation = mol.get_mol_representation()
    
    fp = representation['fingerprint']
    for i in fp:
        bit = fp[i]
        features['f' + str(i)].append(bit)
    descriptors = representation['descriptors']
    for k, v in descriptors.items():
        features[k].append(v)

for k, v in features.items():
    data[k] = pd.Series(v)

  0%|          | 0/304 [00:00<?, ?it/s]

In [9]:
data

,smiles,active,f1,f0,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC(=O)C1=CC2=CC=CC=C2OC1=O,1,0,1,11.287133,-0.572963,11.287133,0.100648,0.507554,188.182,...,0,0,0,0,0,0,0,0,0,0
1,C1COC2=CC=CC=C2C1O,1,0,1,9.481343,-0.333380,9.481343,0.333380,0.606947,150.177,...,0,0,0,0,0,0,0,0,0,0
2,C1=CC=C2C(=C1)C(=CC(=O)O2)O,1,0,1,10.777596,-0.535926,10.777596,0.039954,0.596945,162.144,...,0,0,0,0,0,0,0,0,0,0
3,C1=CC(=CC=C1C2=COC3=C(C2=O)C=CC(=C3O)O)O,1,0,1,12.362099,-0.461244,12.362099,0.057844,0.591055,270.240,...,0,0,0,0,0,0,0,0,0,0
4,C1CC1NC2=C3C(=NC(=N2)N)N(C=N3)[C@@H]4C[C@@H](C...,1,0,1,9.257136,0.162245,9.257136,0.162245,0.727230,286.339,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,C1CC(CCC1CN)C(=O)O,0,0,1,10.518101,-0.638452,10.518101,0.099259,0.623852,157.213,...,0,0,0,0,0,0,0,0,0,0
300,CN(C)CC(C1=CC=C(C=C1)OC)C2(CCCCC2)O,0,0,1,11.101927,-0.558424,11.101927,0.170098,0.897682,277.408,...,0,0,0,0,0,0,0,0,0,0
301,C1C[C@@H](O[C@@H]1CO)N2C=CC(=NC2=O)N,0,0,1,11.451388,-0.417972,11.451388,0.020187,0.687546,211.221,...,0,0,0,0,0,0,0,0,0,0
302,CC(=O)N[C@@H]1[C@H](C=C(O[C@H]1[C@@H]([C@@H](C...,0,0,1,11.358097,-1.697594,11.358097,0.371123,0.188042,332.313,...,0,0,0,0,0,0,0,0,0,0


In [20]:
data = data.dropna()

In [21]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

y = data[TARGET_COLUMN]

X = data.drop(columns = [TARGET_COLUMN, 'smiles'])
X = MinMaxScaler().fit_transform(X) # added the scaler

X_train_val,  X_test, y_train_val, y_test = train_test_split(X, y, train_size = 0.8, random_state = 42, shuffle = True, stratify = None)

skf = StratifiedKFold(n_splits=4,  random_state=42, shuffle=True)

## Models creating

In [22]:
def get_model(model):
    '''
    Get the model from six state-of-the-art machine learning models.
    '''
    if model=='svm':
        from sklearn.svm import SVC
        names = ["Linear SVM"]
        classifiers = [
        SVC()    
        ]
    elif model=='ab':
        from sklearn.ensemble import AdaBoostClassifier
        names = ["AdaBoost"]
        classifiers = [
        AdaBoostClassifier() 
        ]
    elif model=='knn':
        from sklearn.neighbors import KNeighborsClassifier
        names = ["K-Nearest Neighbors"]
        classifiers = [
        KNeighborsClassifier()
        ]
    elif model=='dt':
        from sklearn.tree import DecisionTreeClassifier
        names = ["Decision Tree"]
        classifiers = [
        DecisionTreeClassifier()   
        ]
    elif model=='nb':
        from sklearn.naive_bayes import GaussianNB
        names = ["Naive Bayes"]
        classifiers = [
         GaussianNB()   
        ]
    elif model=='rfc':
        from sklearn.ensemble import RandomForestClassifier
        names = ["Random Forest"]
        classifiers = [
        RandomForestClassifier()
        ]
    elif model=='xgboost':
        from xgboost import XGBClassifier
        names = ["XGBoost"]
        classifiers = [
        XGBClassifier()
        ]
    else:
        raise RuntimeError('Unknown classifier')
    
    return classifiers

## Parameters list

In [23]:
parameters = {
              'svm': {'model__C': (1, 5, 10, 50, 100), 'model__probability': [True]}, 
              'ab': {'model__n_estimators': (10, 25, 50, 100)}, 
              'knn': {'model__n_neighbors': (3, 5, 10, 50), 'model__leaf_size': (10, 25, 40), 
                      'model__weights': ['uniform', 'distance']}, 
              'dt': {'model__max_depth': (10, 25, 50, 100), 'model__min_samples_leaf': (1, 3, 5, 10)}, 
              'nb': {}, 
              'rfc': {'model__max_depth': (2, 3, 5, 7, 10), 'model__n_estimators': (50, 100, 150, 200),
                     'model__min_samples_leaf': (1, 3, 5, 10)}, 
              'xgboost': {'model__max_depth': (10, 20, 30, 50)}
            }

## Evaluating

In [24]:
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

In [25]:
best_clfs_encode = {}
for name, param in parameters.items():
    model = get_model(name)[0]

    steps = [('model', model)]

    pipeline = Pipeline(steps=steps)
    clf = GridSearchCV(pipeline, param, scoring='roc_auc', cv=skf, n_jobs=-1)
    clf.fit(X_train_val, y_train_val)

    print(f'Best params for {name}:', clf.best_params_)
    pipe = clf.best_estimator_['model']

    print('Test ROC AUC for the best model %.2f' % roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1]))
    print('Test accuracy for the best model %.2f' % accuracy_score(y_test, pipe.predict(X_test)))
    print('Test f1-score for the best model %.2f' % f1_score(y_test, pipe.predict(X_test)))
    print()
    best_clfs_encode[name] = clf.best_estimator_

Best params for svm: {'model__C': 1, 'model__probability': True}
Test ROC AUC for the best model 0.84
Test accuracy for the best model 0.77
Test f1-score for the best model 0.85

Best params for ab: {'model__n_estimators': 10}
Test ROC AUC for the best model 0.78
Test accuracy for the best model 0.75
Test f1-score for the best model 0.85

Best params for knn: {'model__leaf_size': 10, 'model__n_neighbors': 50, 'model__weights': 'distance'}
Test ROC AUC for the best model 0.78
Test accuracy for the best model 0.74
Test f1-score for the best model 0.84

Best params for dt: {'model__max_depth': 100, 'model__min_samples_leaf': 5}
Test ROC AUC for the best model 0.71
Test accuracy for the best model 0.74
Test f1-score for the best model 0.83

Best params for nb: {}
Test ROC AUC for the best model 0.67
Test accuracy for the best model 0.38
Test f1-score for the best model 0.34

Best params for rfc: {'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__n_estimators': 100}
Test ROC AUC

/home/lizzka239/anaconda3/envs/rdkit/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
